In [99]:
import os
import shutil
import random
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import albumentations as A
from ultralytics import YOLO

# Dataset Generation

In [209]:
MIAS_IMG_DIR = "mias"                      # contains /B, /M
LABELS_PATH = "mias_info/labels.txt"

YOLO_IMG_ROOT = "YOLO_detection/datasets/images"
YOLO_LABEL_ROOT = "YOLO_detection/datasets/labels"
os.makedirs(f"{YOLO_IMG_ROOT}/train", exist_ok=True)
os.makedirs(f"{YOLO_IMG_ROOT}/val", exist_ok=True)
os.makedirs(f"{YOLO_LABEL_ROOT}/train", exist_ok=True)
os.makedirs(f"{YOLO_LABEL_ROOT}/val", exist_ok=True)

SEVERITY_TO_CLASS = {"M": 0, "B": 1}

cols = ['refno', 'tissue', 'class', 'severity', 'x', 'y', 'r']
df = pd.read_csv(LABELS_PATH, sep=r"\s+", names=cols)
df = df[df['severity'].isin(SEVERITY_TO_CLASS.keys())].dropna(subset=['x', 'y', 'r'])

df['image_id'] = df['refno'].str.extract(r'(mdb\d{3})')
df['class_id'] = df['severity'].map(SEVERITY_TO_CLASS).astype(int)

In [225]:
df.head()

,refno,tissue,class,severity,x,y,r,image_id,class_id
0,mdb001,G,CIRC,B,535,425,197.0,mdb001,1
1,mdb002,G,CIRC,B,522,280,69.0,mdb002,1
4,mdb005,F,CIRC,B,477,133,30.0,mdb005,1
5,mdb005,F,CIRC,B,500,168,26.0,mdb005,1
10,mdb010,F,CIRC,B,525,425,33.0,mdb010,1


In [227]:
df.tail()

,refno,tissue,class,severity,x,y,r,image_id,class_id
281,mdb274,F,MISC,M,127,505,123.0,mdb274,0
297,mdb290,D,CIRC,B,337,353,45.0,mdb290,1
319,mdb312,F,MISC,B,240,263,20.0,mdb312,1
321,mdb314,F,MISC,B,518,191,39.0,mdb314,1
322,mdb315,D,CIRC,B,516,447,93.0,mdb315,1


In [211]:
unique_ids = df['image_id'].unique()
train_ids, val_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)

In [213]:
def convert_bbox(x, y, r, w, h):
    return [
        x / w,
        y / h,
        (2 * r) / w,
        (2 * r) / h
    ]

In [215]:
AUGMENT = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Resize(512, 512),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

In [217]:
AUGMENTATION_COUNT = 3

In [219]:
def process(split_ids, split_name):
    for img_id in tqdm(split_ids, desc=f"Processing {split_name}"):
        rows = df[df['image_id'] == img_id]
        if rows.empty:
            continue

        folder = rows.iloc[0]['severity']
        image_path = os.path.join(MIAS_IMG_DIR, folder, img_id + '.png')
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image: {image_path}")
            continue
        h, w = image.shape[:2]

        boxes = []
        class_labels = []
        for _, row in rows.iterrows():
            x, y, r = float(row['x']), float(row['y']), float(row['r'])
            cls = int(row['class_id'])
            boxes.append(convert_bbox(x, y, r, w, h))
            class_labels.append(cls)

        out_img_path = os.path.join(YOLO_IMG_ROOT, split_name, img_id + ".png")
        out_label_path = os.path.join(YOLO_LABEL_ROOT, split_name, img_id + ".txt")
        cv2.imwrite(out_img_path, image)
        with open(out_label_path, 'w') as f:
            for box, cls in zip(boxes, class_labels):
                f.write(f"{cls} " + " ".join([f"{v:.6f}" for v in box]) + "\n")

        for i in range(AUGMENTATION_COUNT):
            transformed = AUGMENT(image=image, bboxes=boxes, class_labels=class_labels)
            aug_img = transformed['image']
            aug_bboxes = transformed['bboxes']
            aug_labels = transformed['class_labels']

            aug_img_name = f"{img_id}_aug{i}"
            aug_img_path = os.path.join(YOLO_IMG_ROOT, split_name, aug_img_name + ".png")
            aug_label_path = os.path.join(YOLO_LABEL_ROOT, split_name, aug_img_name + ".txt")
            cv2.imwrite(aug_img_path, aug_img)
            with open(aug_label_path, 'w') as f:
                for box, cls in zip(aug_bboxes, aug_labels):
                    f.write(f"{cls} " + " ".join([f"{v:.6f}" for v in box]) + "\n")

In [221]:
process(train_ids, "train")
process(val_ids, "val")

Processing val: 100%|██████████| 23/23 [00:04<00:00,  5.57it/s]


# Training with YOLOv8

In [167]:
model_s = YOLO('yolov8s.pt')
model_s.train(
    data="C:/Users/seren/bioe486_project/YOLO_detection/datasets/data.yaml", 
    epochs=45, 
    imgsz=768, 
    batch=8, 
    name="yolo_train"
)

100%|██████████| 21.5M/21.5M [00:02<00:00, 10.4MB/s]


Ultralytics 8.3.133  Python-3.11.7 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i7-1355U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/seren/bioe486_project/YOLO_detection/datasets/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=45, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pe

train: Scanning C:\Users\seren\bioe486_project\YOLO_detection\datasets\labels\train.cache... 88 images, 0 backgrounds, 0 corrupt: 100%|██████████| 88/88 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.30.1 ms, read: 173.550.6 MB/s, size: 241.5 KB)



val: Scanning C:\Users\seren\bioe486_project\YOLO_detection\datasets\labels\val.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]


...

                   all         23         23    0.00118       0.05     0.0243    0.00729



45 epochs completed in 2.114 hours.
Optimizer stripped from runs\detect\yolo_train11\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\yolo_train11\weights\best.pt, 22.5MB

Validating runs\detect\yolo_train11\weights\best.pt...
Ultralytics 8.3.133  Python-3.11.7 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i7-1355U)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:12<00:00,  6.22s/it]


                   all         23         23    0.00101       0.05      0.048     0.0192
             malignant         10         10    0.00201        0.1     0.0959     0.0384
                benign         13         13          0          0          0          0
Speed: 9.2ms preprocess, 493.0ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\yolo_train11


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000257571EEB10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [272]:
# augnmented data 
model_n = YOLO('yolov8n.pt')
model_n.train(
    data="C:/Users/seren/bioe486_project/YOLO_detection/datasets/data.yaml", 
    epochs=100,                
    imgsz=256,                
    batch=16,
    name="yolo_train",
    device="cpu",             
    workers=2,
    patience=50,             
    verbose=True
)


KeyboardInterrupt

